<a href="https://colab.research.google.com/github/Praneeth33/Hello_world-/blob/main/friend_recommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing Spark

In [ ]:
!pip install pyspark

#Importing Libraries

In [ ]:
import sys
import itertools
from pyspark import SparkConf, SparkContext


#Line to Friend Ownership

In [ ]:
def line_to_friend_ownership(line):
   
    split = line.split()
    user_id = int(split[0])

    if len(split) == 1:
        friends = []
    else:
        friends = list(map(lambda x: int(x), split[1].split(',')))

    return user_id, friends


#Friend Ownership Connection

In [ ]:
def friend_ownership_to_connection(f_o):
    user_id = f_o[0]
    friends = f_o[1]

    connections = []

    for friend_id in friends:
        key = (user_id, friend_id)
        if user_id > friend_id:
            key = (friend_id, user_id)

        connections.append(
            (key, 0)
        )

    for friend_pair in itertools.combinations(friends, 2):
        friend_0 = friend_pair[0]
        friend_1 = friend_pair[1]

        key = (friend_0, friend_1)
        if friend_0 > friend_1:
            key = (friend_1, friend_0)
        connections.append(
            (key, 1)
        )

    return connections


#Mutual Friend Count to Recommendation

In [ ]:


def mutual_friend_count_to_recommendation(m):
    connection = m[0]
    count = m[1]

    friend_0 = connection[0]
    friend_1 = connection[1]

    recommendation_0 = (friend_0, (friend_1, count))
    recommendation_1 = (friend_1, (friend_0, count))

    return [recommendation_0, recommendation_1]


#Recommendaion to sorted truncated

In [ ]:
def recommendation_to_sorted_truncated(recs):
    if len(recs) > 1024:
        max_indices = []

        for current_rec_number in range(0, 10):
            current_max_index = 0
            for i in range(1, len(recs)):
                rec = recs[i]
                if rec[1] >= recs[current_max_index][1] and i not in max_indices:
                    current_max_index = i

            max_indices.append(current_max_index)

        recs = [recs[i] for i in max_indices]

    recs.sort(key=lambda x: (-x[1], x[0]))
    return list(map(lambda x: x[0], recs))[:10]



#Main Funcion

In [ ]:
lines = sc.textFile("FriendData.txt")

friend_ownership = lines.map(line_to_friend_ownership)
friend_edges = friend_ownership.flatMap(friend_ownership_to_connection)
friend_edges.cache()

mutual_friend_counts = friend_edges.groupByKey() \
    .filter(lambda edge: 0 not in edge[1]) \
    .map(lambda edge: (edge[0], sum(edge[1])))

recommendations = mutual_friend_counts.flatMap(mutual_friend_count_to_recommendation) \
    .groupByKey() \
    .map(lambda m: (m[0], recommendation_to_sorted_truncated(list(m[1]))))

recommendations.saveAsTextFile("Result")
sc.stop()